<a href="https://colab.research.google.com/github/LizzyZhang-tutu/NLP_Learning/blob/master/machineLearningSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Naive Bayes

import pandas as pd
movies = pd.read_csv('movies.csv')
movies.head().round(2)



,id,sentiment,text
0,1,2.27,The Rock is destined to be the 21st Century's ...
1,2,3.53,The gorgeously elaborate continuation of ''The...
2,3,-0.60,Effective but too tepid biopic
3,4,1.47,If you sometimes like to go to the movies to h...
4,5,1.73,"Emerges as something rare, an issue movie that..."


In [3]:
movies.describe().round(2)

,id,sentiment
count,10605.00,10605.00
mean,5303.00,0.00
std,3061.54,1.92
min,1.00,-3.88
25%,2652.00,-1.77
50%,5303.00,-0.08
75%,7954.00,1.83
max,10605.00,3.94


In [6]:
pd.set_option('display.width', 75)
from nltk.tokenize import casual_tokenize
from collections import Counter

bags_of_words = []
for text in movies.text:
  bags_of_words.append(Counter(casual_tokenize(text)))

df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)
df_bows.shape

(10605, 20756)

In [7]:
df_bows.head()

,The,Rock,is,destined,to,be,the,21st,Century's,new,',Conan,and,that,he's,going,make,a,splash,even,greater,than,Arnold,Schwarzenegger,",",Jean,Claud,Van,Damme,or,Steven,Segal,.,gorgeously,elaborate,continuation,of,Lord,Rings,trilogy,...,Overwrought,snooze,Feeble,salaciously,Disjointed,humbuggery,Eh,unrealistic,nrelentingly,Painfully,Grating,Dramatically,Predictably,Arty,Incoherence,reigns,assed,Abysmally,Bland,ame,drudgery,snubbing,Mildly,Terrible,Degenerates,hogwash,Crummy,Wishy,Inconsequential,Insufferably,Ill,slummer,Rashomon,dipsticks,Bearable,Staggeringly,’,ve,muttering,dissing
0,1,1,1,1,2,1,1,1,1,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,0,1,0,0,0,4,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,1,1,4,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,4,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# find the keywords that predict sentiment from natural language text
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb = nb.fit(df_bows, movies.sentiment > 0)
movies['predicted_sentiment'] = nb.predict(df_bows) * 8 - 4
movies['error'] = (movies.predicted_sentiment - movies.sentiment).abs()

print(movies.error.mean())

2.3911742904638262


In [17]:
# read products data

products = pd.read_csv('products.csv')
product_bags_of_words = []
for text in products.text:
  product_bags_of_words.append(Counter(casual_tokenize(text)))
df_product_bows = pd.DataFrame.from_records(product_bags_of_words)
df_product_bows = df_product_bows.fillna(0).astype(int)
df_all_bows = df_bows.append(df_product_bows)

df_all_bows.columns

Index(['The', 'Rock', 'is', 'destined', 'to', 'be', 'the', '21st',
       'Century's', 'new',
       ...
       'sligtly', 'owner', '81', 'defectively', 'warrranty', 'expire',
       'expired', 'voids', 'baghdad', 'harddisk'],
      dtype='object', length=23302)

In [18]:
df_product_bows = df_all_bows.iloc[len(movies):][df_bows.columns]
df_product_bows.shape

(3546, 20756)

In [19]:
df_bows.shape

(10605, 20756)

In [20]:
import numpy as np
df_product_bows[:] = np.nan_to_num(df_product_bows)
products['ispos'] = (products.sentiment > 0).astype(int)
products['pred'] = nb.predict(df_product_bows.values).astype(int)
products.head()

,id,sentiment,text,ispos,pred
0,1_1,-0.90,troubleshooting ad-2500 and ad-2600 no picture...,0,0
1,1_2,-0.15,"repost from january 13, 2004 with a better fit...",0,0
2,1_3,-0.20,does your apex dvd player only play dvd audio ...,0,0
3,1_4,-0.10,or does it play audio and video but scrolling ...,0,0
4,1_5,-0.50,before you try to return the player or waste h...,0,0


In [22]:
(products.pred == products.ispos).sum() / len(products)

0.5572476029328821

The Naive Bayes model doesn't deal with negation as well as Vader does. So you would need to incorporate n-grams into your token to connect negation words such as not or never to the positive words they might be used to qualify.